# L1 언어 모델, 채팅 형식, 그리고 토큰
## Setup
#### API key와 관련된 파이썬 라이브러리를 불러옵니다.
본 강의는 당신을 위한 OpenAI API key를 불러오는 코드를 제공합니다.  
(공짜라는 뜻 💰❌)

In [ ]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

위 방식처럼 여러분의 API key를 직접 변수에 할당하지 않고 다른 파일에서 불러오는 방식이 권장됩니다.  
<span style='color:red'>단순히 파일이 공유되거나 깃허브에 업로드되는 것만으로 API key가 유출되어 큰 피해를 입을 수 있기 때문</span>입니다.  
꼭 저런 파일이 아니어도 괜찮으니 편한대로 쓰면 됩니다.  
저같은 경우는 txt 파일에 키를 저장해두고 불러와서 썼습니다.

#### helper function
아마 "ChatGPT Prompt Engineering for Developers" 강의를 들으셨다면 익숙한 기능이겠네요!  
주어진 입력 다음을 완성하는 출력을 반환하는 함수입니다.

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

## 모델에게 질문하고 완성된 문장 반환받기

위에서 정의한 함수를 사용하고 "What is the capital of France?" 라는 질문을 입력으로 주었습니다.  
이에 대한 <span style='color:red'>모델의 답변이 response에 저장</span>되는 것을 확인할 수 있습니다.

In [ ]:
response = get_completion("What is the capital of France?")

In [ ]:
print(response)

## 토큰

아래 코드를 실행시켜보면 생각보다 간단해 보이는 이 태스크를 모델이 제대로 수행하지 못한다는 것을 확인할 수 있습니다.  
이는 모델이 언어를 입력으로 받을 때, 학습했던 것들을 기준으로 단어를 쪼개어 토큰화하기 때문입니다.  
예를 들면 lollipop은 lol,li,pop 등으로 쪼개지기 때문에 <span style='color:red'>우리의 의도대로 모델이 입력을 받지 못하는 것</span>입니다.

In [ ]:
response = get_completion("Take the letters in lollipop \
and reverse them")
print(response)

"lollipop" 을 거꾸로하면 "popillol" 이렇게 되어야겠죠.  
따라서 입력이 제대로 구분될 수 있도록 아래와 같이 입력을 제공하면 원하는 결과를 얻을 수 있습니다.

In [ ]:
response = get_completion("""Take the letters in \
l-o-l-l-i-p-o-p and reverse them""")

In [ ]:
response

## Helper function (채팅 형태)
여기에 우리가 강의에서 사용할 새로운 helper function이 있습니다.  
이전과 다르게 messages라는 파라미터를 지정하여 <span style='color:red'>여러 개의 문장을 입력으로 받게 된다</span>는 것을 알 수 있습니다.

In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message["content"]


role을 보면 <span style='color:red'>system은 말 그대로 시스템적인 세팅이고, user는 사용자가 입력한 문장</span>을 뜻하게 됩니다.  
아래 예시에서는 assistant 역할과, 한 문장으로 답변하라는 세팅 두 개를 각각 사용한 경우와 둘을 합친 경우를 보여줍니다.

In [ ]:
messages =  [  
{'role':'system', 
 'content':"""You are an assistant who\
 responds in the style of Dr Seuss."""},    
{'role':'user', 
 'content':"""write me a very short poem\
 about a happy carrot"""},  
] 
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
# length
messages =  [  
{'role':'system',
 'content':'All your responses must be \
one sentence long.'},    
{'role':'user',
 'content':'write me a story about a happy carrot'},  
] 
response = get_completion_from_messages(messages, temperature =1)
print(response)

In [ ]:
# combined
messages =  [  
{'role':'system',
 'content':"""You are an assistant who \
responds in the style of Dr Seuss. \
All your responses must be one sentence long."""},    
{'role':'user',
 'content':"""write me a story about a happy carrot"""},
] 
response = get_completion_from_messages(messages, 
                                        temperature =1)
print(response)

모델이 반환하는 결과를 담는 변수 response의 값들을 꺼내어 보면 사용된 토큰의 개수를 알 수 있습니다.  
일반적으로 GPT는 입력을 최대 4,000 토큰 길이로 받으며, 출력으로 반환하는 토큰의 개수 단위로 금액을 지불하게 설정되어 있습니다.  
짧은 문장들은 괜찮지만 아무 생각없이 긴 문장들을 주-욱 복붙하면 순식간에 텅-장이 될 수 있습니다 ⚠

In [ ]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0, 
                                   max_tokens=500):
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message["content"]
    
    token_dict = {
'prompt_tokens':response['usage']['prompt_tokens'],
'completion_tokens':response['usage']['completion_tokens'],
'total_tokens':response['usage']['total_tokens'],
    }

    return content, token_dict

In [ ]:
messages = [
{'role':'system', 
 'content':"""You are an assistant who responds\
 in the style of Dr Seuss."""},    
{'role':'user',
 'content':"""write me a very short poem \ 
 about a happy carrot"""},  
] 
response, token_dict = get_completion_and_token_count(messages)

In [ ]:
print(response)

In [ ]:
print(token_dict)

#### OpenAI API를 수업(사이트) 외부에서 사용할 때 주의할 점

OepnAI 파이썬 라이브러리 설치 방법:
```
!pip install openai
```

해당 라이브러리를 사용하기 위해서는 각자의 계정에 부여된 비밀 key를 사용해야 합니다. 이는 다음 링크에서 확인 가능합니다. [website](https://platform.openai.com/account/api-keys)  
참고로 API key는 발급받을 때만 알려주고 그 이후로는 <span style='color:red'>절대로 다시 확인하지 못하는 구조</span>이기 때문에, 혼자만 확인 가능한 공간에 잘 기록해두는 것이 중요합니다.  

처음에 말씀드렸던 것처럼 OpenAI API key를 다른 곳에 저장해두고 라이브러리를 사용하여 불러오는 방법을 취할 수도 있습니다:
 ```
 !export OPENAI_API_KEY='sk-...'
 ```

아니면 변수에 값을 직접 할당할 수도 있죠:

```
import openai
openai.api_key = "sk-..."
```